# Лабораторная работа №2: Выделение признаков изображений

## Цель работы
Изучение методов выделения признаков на изображениях: обнаружение линий и окружностей с помощью преобразования Хафа, а также сегментация изображений на основе текстурных признаков.

## Выполненные задачи

### a. Выделение прямых и окружностей
1. Обнаружение прямых линий с использованием преобразования Хафа (HoughLinesP)
2. Обнаружение окружностей с использованием преобразования Хафа (HoughCircles)

### c. Сегментация по текстуре
1. Реализация алгоритма роста регионов (Region Growing)
2. Использование локальных бинарных паттернов (LBP) для анализа текстуры
3. Комбинированный подход: анализ цвета и текстуры для точной сегментации


In [ ]:
# Импорт необходимых библиотек
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.feature import local_binary_pattern
from collections import deque
import os

# Настройка отображения
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['font.size'] = 10
print("✓ Библиотеки успешно импортированы")


## Функции обработки изображений

Ниже представлены все функции для выделения признаков, с подробными комментариями.


In [ ]:
def find_hough_lines(image):
    """
    Обнаружение прямых линий на изображении с использованием преобразования Хафа.
    
    Алгоритм:
    1. Преобразование изображения в оттенки серого (если необходимо)
    2. Применение детектора краёв Canny
    3. Применение вероятностного преобразования Хафа (HoughLinesP)
    4. Отрисовка найденных линий на исходном изображении
    
    Параметры:
        image: входное изображение (BGR или серое)
    
    Возвращает:
        output_image: изображение с отмеченными линиями (зелёный цвет)
    """
    # Преобразуем в серое, если изображение цветное
    if len(image.shape) == 3:
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray_image = image.copy()
    
    # Применяем детектор краёв Canny
    # Параметры: нижний порог = 50, верхний порог = 150
    edges = cv2.Canny(gray_image, 50, 150, apertureSize=3)
    
    # Применяем вероятностное преобразование Хафа
    # Параметры:
    # - rho = 1: разрешение по расстоянию в пикселях
    # - theta = pi/180: разрешение по углу в радианах (1 градус)
    # - threshold = 100: минимальное количество пересечений для обнаружения линии
    # - minLineLength = 100: минимальная длина линии
    # - maxLineGap = 10: максимальный разрыв между сегментами линии
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, 100, minLineLength=100, maxLineGap=10)
    
    # Создаём копию исходного изображения для отрисовки
    output_image = image.copy()
    
    # Отрисовываем найденные линии
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(output_image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        print(f"Найдено линий: {len(lines)}")
    else:
        print("Линии не найдены")
    
    return output_image


def find_hough_circles(image):
    """
    Обнаружение окружностей на изображении с использованием преобразования Хафа.
    
    Алгоритм:
    1. Преобразование изображения в оттенки серого
    2. Предобработка: размытие по Гауссу для снижения шума
    3. Адаптивная эквализация гистограммы (CLAHE) для улучшения контраста
    4. Применение преобразования Хафа для окружностей
    5. Отрисовка найденных окружностей и их центров
    
    Параметры:
        image: входное изображение (BGR или серое)
    
    Возвращает:
        output_image: изображение с отмеченными окружностями (зелёный) и центрами (красный)
    """
    # Преобразуем в серое, если изображение цветное
    if len(image.shape) == 3:
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray_image = image.copy()
    
    # Предобработка: размытие для снижения шума
    img_blur = cv2.GaussianBlur(gray_image, (9, 9), 2)
    
    # Адаптивная эквализация гистограммы для улучшения контраста
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    img_enhanced = clahe.apply(img_blur)
    
    # Применяем преобразование Хафа для окружностей
    # Параметры:
    # - method = HOUGH_GRADIENT: метод обнаружения
    # - dp = 1: соотношение разрешения аккумулятора к разрешению изображения
    # - minDist = 30: минимальное расстояние между центрами окружностей
    # - param1 = 100: верхний порог для детектора Canny
    # - param2 = 30: порог для центра окружности (чем меньше, тем больше ложных срабатываний)
    # - minRadius = 5: минимальный радиус окружности
    # - maxRadius = 300: максимальный радиус окружности
    circles = cv2.HoughCircles(img_enhanced, cv2.HOUGH_GRADIENT, 
                               dp=1, 
                               minDist=30,
                               param1=100,
                               param2=30,
                               minRadius=5, 
                               maxRadius=300)
    
    # Создаём копию исходного изображения для отрисовки
    output_image = image.copy()
    
    # Отрисовываем найденные окружности
    if circles is not None:
        circles = np.uint16(np.around(circles))
        for i in circles[0, :]:
            # Рисуем окружность (зелёным)
            cv2.circle(output_image, (i[0], i[1]), i[2], (0, 255, 0), 2)
            # Рисуем центр (красным)
            cv2.circle(output_image, (i[0], i[1]), 2, (0, 0, 255), 3)
        print(f"Найдено окружностей: {len(circles[0])}")
    else:
        print("Окружности не найдены")
    
    return output_image


def segment_by_texture(image, seed_x, seed_y):
    """
    Сегментация изображения по текстуре с использованием алгоритма роста регионов.
    
    Алгоритм:
    1. Преобразование изображения в оттенки серого
    2. Уменьшение разрешения для ускорения обработки (опционально)
    3. Вычисление локальных бинарных паттернов (LBP) для всего изображения
    4. Извлечение целевого патча вокруг затравочной точки
    5. Вычисление гистограммы LBP для целевого патча
    6. Алгоритм роста регионов:
       - Быстрая проверка по цветовому сходству
       - Медленная проверка по текстурному сходству (только для похожих по цвету)
    7. Масштабирование маски обратно к исходному размеру
    8. Морфологическая обработка для сглаживания результата
    9. Наложение результата на исходное изображение
    
    Параметры:
        image: входное BGR изображение
        seed_x: X координата затравочной точки
        seed_y: Y координата затравочной точки
    
    Возвращает:
        output_image: изображение с выделенным регионом (зелёным цветом)
        success: булево значение, True если сегментация успешна
    """
    # Преобразуем в серое
    grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    h, w = grayscale_image.shape
    
    # Уменьшаем разрешение для ускорения (если изображение большое)
    scale_factor = 2 if max(h, w) > 800 else 1
    small_gray = cv2.resize(grayscale_image, (w // scale_factor, h // scale_factor))
    small_color = cv2.resize(image, (w // scale_factor, h // scale_factor))
    
    # Пересчитываем координаты seed
    sx, sy = seed_x // scale_factor, seed_y // scale_factor
    sh, sw = small_gray.shape
    
    # Вычисляем LBP (Local Binary Patterns) для всего изображения
    # LBP - это метод описания текстуры, который кодирует локальную структуру
    radius = 1
    n_points = 8 * radius  # Количество соседних точек
    lbp = local_binary_pattern(small_gray, n_points, radius, 'uniform')
    
    # Размер патча для анализа текстуры
    patch_size = 9
    half_patch = patch_size // 2
    
    # Проверяем, что seed point не слишком близко к краю
    if not (half_patch <= sx < sw - half_patch and half_patch <= sy < sh - half_patch):
        print("Ошибка: точка слишком близко к краю изображения")
        return image.copy(), False
    
    # Извлекаем целевой патч вокруг затравочной точки
    seed_patch_lbp = lbp[sy - half_patch: sy + half_patch + 1,
                         sx - half_patch: sx + half_patch + 1]
    
    # Вычисляем гистограмму LBP для целевого патча
    target_lbp_hist, _ = np.histogram(seed_patch_lbp, bins=n_points + 2, 
                                      range=(0, n_points + 2), density=True)
    
    # Извлекаем цвет затравочной точки для быстрой фильтрации
    seed_color = small_color[sy, sx].astype(np.float32)
    
    # Инициализируем маску и очередь для алгоритма роста регионов
    mask = np.zeros((sh, sw), dtype=np.uint8)
    queue = deque([(sx, sy)])
    mask[sy, sx] = 255
    
    # Пороговые значения для сходства
    color_threshold = 40  # Максимальная разница по цвету (евклидово расстояние)
    texture_threshold = 0.5  # Минимальная корреляция текстуры (от -1 до 1)
    
    # Алгоритм роста регионов
    processed_pixels = 0
    while queue:
        x, y = queue.popleft()
        
        # Проверяем 4-связных соседей (вверх, вниз, влево, вправо)
        for dx, dy in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
            nx, ny = x + dx, y + dy
            
            # Проверяем границы и посещение
            if not (half_patch <= nx < sw - half_patch and half_patch <= ny < sh - half_patch):
                continue
            if mask[ny, nx] != 0:
                continue
            
            # БЫСТРАЯ проверка: сходство по цвету
            current_color = small_color[ny, nx].astype(np.float32)
            color_diff = np.linalg.norm(current_color - seed_color)
            
            if color_diff > color_threshold:
                continue  # Слишком разные по цвету - пропускаем
            
            # МЕДЛЕННАЯ проверка: сходство по текстуре (только для похожих по цвету)
            current_patch_lbp = lbp[ny - half_patch: ny + half_patch + 1,
                                    nx - half_patch: nx + half_patch + 1]
            current_lbp_hist, _ = np.histogram(current_patch_lbp, bins=n_points + 2,
                                                range=(0, n_points + 2), density=True)
            
            # Сравниваем гистограммы с помощью корреляции
            texture_score = cv2.compareHist(target_lbp_hist.astype(np.float32),
                                            current_lbp_hist.astype(np.float32),
                                            cv2.HISTCMP_CORREL)
            
            # Если текстура похожа, добавляем пиксель в регион
            if texture_score > texture_threshold:
                mask[ny, nx] = 255
                queue.append((nx, ny))
                processed_pixels += 1
    
    print(f"Обработано пикселей: {processed_pixels}")
    
    # Масштабируем маску обратно к исходному размеру
    if scale_factor > 1:
        mask = cv2.resize(mask, (w, h), interpolation=cv2.INTER_NEAREST)
    
    # Морфологическая обработка для сглаживания
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)  # Закрываем дыры
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)   # Убираем шум
    
    # Накладываем результат на исходное изображение
    output_image = image.copy()
    output_image[mask == 255] = [0, 255, 0]  # Зелёный цвет для выделенного региона
    
    return output_image, True

print("✓ Все функции обработки определены")


## Вспомогательные функции для визуализации


In [ ]:
def display_comparison(original, processed, title):
    """
    Отображает исходное и обработанное изображения рядом.
    
    Параметры:
        original: исходное изображение (BGR)
        processed: обработанное изображение (BGR)
        title: заголовок
    """
    fig, axes = plt.subplots(1, 2, figsize=(15, 7))
    fig.suptitle(title, fontsize=16, fontweight='bold')
    
    # Исходное изображение
    original_rgb = cv2.cvtColor(original, cv2.COLOR_BGR2RGB)
    axes[0].imshow(original_rgb)
    axes[0].set_title('Исходное изображение', fontsize=12)
    axes[0].axis('off')
    
    # Обработанное изображение
    processed_rgb = cv2.cvtColor(processed, cv2.COLOR_BGR2RGB)
    axes[1].imshow(processed_rgb)
    axes[1].set_title('Результат обработки', fontsize=12)
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()

print("✓ Функции визуализации определены")


## Загрузка тестовых изображений

Для тестирования используются три различных изображения. Поместите ваши изображения в папку `test_images/` с именами `test1.jpg`, `test2.jpg`, `test3.jpg`.


In [ ]:
# Создаём папку для тестовых изображений, если её нет
os.makedirs('test_images', exist_ok=True)

# Загружаем тестовые изображения
test_images = []
image_paths = ['test_images/test1.jpg', 'test_images/test2.jpg', 'test_images/test3.jpg']

for path in image_paths:
    if os.path.exists(path):
        img = cv2.imread(path)
        if img is not None:
            test_images.append(img)
            print(f"Загружено: {path}, размер: {img.shape}")

if len(test_images) == 0:
    print("\n⚠️ ТЕСТОВЫЕ ИЗОБРАЖЕНИЯ НЕ НАЙДЕНЫ!")
    print("Создайте папку 'test_images' и поместите в неё файлы test1.jpg, test2.jpg, test3.jpg")
    print("\nДля демонстрации создадим синтетические изображения...")
    
    # Изображение 1: Линии и геометрические фигуры
    img1 = np.ones((400, 600, 3), dtype=np.uint8) * 255
    cv2.rectangle(img1, (100, 100), (250, 250), (100, 100, 100), -1)
    cv2.line(img1, (50, 50), (550, 100), (0, 0, 0), 3)
    cv2.line(img1, (100, 300), (500, 350), (0, 0, 0), 3)
    cv2.line(img1, (300, 50), (350, 380), (0, 0, 0), 3)
    
    # Изображение 2: Окружности
    img2 = np.ones((400, 600, 3), dtype=np.uint8) * 255
    cv2.circle(img2, (150, 150), 60, (50, 50, 50), -1)
    cv2.circle(img2, (400, 200), 80, (50, 50, 50), -1)
    cv2.circle(img2, (300, 320), 50, (50, 50, 50), -1)
    
    # Изображение 3: Текстуры для сегментации
    img3 = np.ones((400, 600, 3), dtype=np.uint8) * 255
    # Регион 1: Вертикальные линии
    for i in range(0, 300, 5):
        cv2.line(img3, (i, 0), (i, 200), (100, 150, 100), 2)
    # Регион 2: Горизонтальные линии
    for i in range(0, 200, 5):
        cv2.line(img3, (300, i), (600, i), (150, 100, 150), 2)
    # Регион 3: Крестообразные линии
    for i in range(200, 400, 10):
        cv2.line(img3, (0, i), (300, i), (100, 100, 150), 1)
        cv2.line(img3, (0, i-5), (300, i-5), (100, 100, 150), 1)
    for j in range(0, 300, 10):
        cv2.line(img3, (j, 200), (j, 400), (100, 100, 150), 1)
    
    test_images = [img1, img2, img3]
    print("✓ Созданы синтетические тестовые изображения")


## Результаты обработки

Ниже представлены результаты применения всех функций к каждому из трёх тестовых изображений.

---

### Изображение 1: Обнаружение линий


In [ ]:
if len(test_images) >= 1:
    img = test_images[0]
    result = find_hough_lines(img)
    display_comparison(img, result, 'Изображение 1: Обнаружение линий методом Хафа')


### Изображение 2: Обнаружение окружностей


In [ ]:
if len(test_images) >= 2:
    img = test_images[1]
    result = find_hough_circles(img)
    display_comparison(img, result, 'Изображение 2: Обнаружение окружностей методом Хафа')


### Изображение 3: Сегментация по текстуре

Для демонстрации выбираем несколько затравочных точек в разных регионах изображения.


In [ ]:
if len(test_images) >= 3:
    img = test_images[2]
    h, w = img.shape[:2]
    
    # Выбираем несколько затравочных точек для демонстрации
    seed_points = [
        (150, 100, "Регион 1"),  # Вертикальные линии
        (450, 100, "Регион 2"),  # Горизонтальные линии
        (150, 300, "Регион 3"),  # Крестообразные линии
    ]
    
    for seed_x, seed_y, region_name in seed_points:
        print(f"\n{region_name}: затравочная точка ({seed_x}, {seed_y})")
        result, success = segment_by_texture(img, seed_x, seed_y)
        if success:
            display_comparison(img, result, f'Изображение 3: Сегментация по текстуре - {region_name}')


### Дополнительное тестирование на всех изображениях

Применим все методы к оставшимся изображениям для полноты тестирования.


In [ ]:
# Тестируем обнаружение окружностей на изображении 1
if len(test_images) >= 1:
    print("\n=== Изображение 1: Обнаружение окружностей ===")
    img = test_images[0]
    result = find_hough_circles(img)
    display_comparison(img, result, 'Изображение 1: Обнаружение окружностей')

# Тестируем обнаружение линий на изображении 2
if len(test_images) >= 2:
    print("\n=== Изображение 2: Обнаружение линий ===")
    img = test_images[1]
    result = find_hough_lines(img)
    display_comparison(img, result, 'Изображение 2: Обнаружение линий')

# Тестируем сегментацию на изображении 1
if len(test_images) >= 1:
    print("\n=== Изображение 1: Сегментация ===")
    img = test_images[0]
    h, w = img.shape[:2]
    # Выбираем точку в центре прямоугольника
    result, success = segment_by_texture(img, 175, 175)
    if success:
        display_comparison(img, result, 'Изображение 1: Сегментация по текстуре')


## Выводы

### Преобразование Хафа для прямых линий

**Принцип работы:**
- Преобразование Хафа переводит задачу обнаружения линий из пространства изображения в параметрическое пространство (ρ, θ)
- Каждая точка на краях изображения может принадлежать множеству линий, образующих синусоиду в пространстве параметров
- Точки пересечения синусоид соответствуют параметрам реальных линий на изображении

**Результаты:**
- Вероятностное преобразование Хафа (HoughLinesP) эффективно находит прямые линии на изображениях
- Метод устойчив к разрывам в линиях благодаря параметру `maxLineGap`
- Параметр `minLineLength` позволяет отфильтровать короткие шумовые сегменты
- Предварительное применение детектора Canny критично для качества результатов

**Ограничения:**
- Требует тщательной настройки параметров для разных типов изображений
- Может пропустить линии с малым контрастом
- Не различает отдельные близко расположенные параллельные линии

### Преобразование Хафа для окружностей

**Принцип работы:**
- Обобщение преобразования Хафа на трёхмерное пространство параметров (x, y, r)
- Метод HOUGH_GRADIENT использует градиенты изображения для поиска центров окружностей
- Аккумулятор накапливает "голоса" за возможные центры окружностей

**Результаты:**
- Метод успешно обнаруживает окружности различного размера
- Предобработка (размытие + CLAHE) значительно улучшает качество обнаружения
- CLAHE (адаптивная эквализация гистограммы) помогает выявить окружности при неравномерном освещении
- Параметр `minDist` эффективно предотвращает множественное обнаружение одной окружности

**Ограничения:**
- Чувствителен к шуму и требует качественной предобработки
- Параметр `param2` требует тонкой настройки: слишком малое значение даёт много ложных срабатываний
- Не обнаруживает эллипсы и деформированные окружности
- Вычислительно затратен для больших изображений

### Сегментация по текстуре с использованием LBP

**Принцип работы:**
- Локальные бинарные паттерны (LBP) кодируют локальную текстуру вокруг каждого пикселя
- Алгоритм роста регионов расширяет регион от затравочной точки, добавляя пиксели с похожей текстурой
- Комбинированный критерий (цвет + текстура) обеспечивает более точную сегментацию

**Результаты:**
- LBP эффективно описывает текстурные паттерны, устойчив к изменениям освещённости
- Двухэтапная фильтрация (быстрая по цвету, медленная по текстуре) значительно ускоряет алгоритм
- Морфологические операции (closing/opening) сглаживают границы и устраняют шум
- Масштабирование изображения позволяет обрабатывать большие изображения за разумное время

**Ограничения:**
- Качество зависит от выбора затравочной точки
- Пороги (color_threshold, texture_threshold) требуют настройки для разных изображений
- Не работает для регионов без выраженной текстуры
- Вычислительно затратен для сложных текстур на больших изображениях

### Общие наблюдения

1. **Предобработка критична**: все методы требуют качественной предобработки (размытие, эквализация, детектор краёв)

2. **Баланс точность/скорость**: уменьшение разрешения для сегментации даёт хорошее соотношение скорости и качества

3. **Настройка параметров**: каждый метод имеет несколько критичных параметров, требующих настройки под конкретную задачу

4. **Комбинирование методов**: использование нескольких признаков (цвет + текстура) даёт более робастные результаты

5. **Практическое применение**:
   - Преобразование Хафа: обнаружение объектов правильной формы в системах технического зрения
   - Сегментация по текстуре: медицинская визуализация, анализ материалов, распознавание объектов
